In [1]:
from lasagne.layers import InputLayer, Conv2DLayer, MaxPool2DLayer, DenseLayer, GlobalPoolLayer, Upscale2DLayer
from lasagne.layers import ElemwiseSumLayer, NonlinearityLayer, SliceLayer, ConcatLayer, ScaleLayer
from lasagne.layers import dropout, batch_norm
from lasagne.nonlinearities import rectify, softmax, sigmoid
from lasagne.init import GlorotNormal, GlorotUniform, HeUniform, HeNormal
from lasagne.objectives import squared_error, categorical_crossentropy, categorical_accuracy, binary_accuracy
import lasagne
import theano.tensor as T
import numpy as np
import random
import theano
import os
import pandas as pd
import cv2
import re
import sys

sys.path.append('..')
import benchmark as bm

from fcn1.adapter import adapter as adapter1

Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, CuDNN 3007)
/usr/local/lib/python2.7/dist-packages/theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


# Utility functions

In [2]:
def build_volume(area, location, resolution):
    def _step(idx_, prior_result_, area_, location_, resolution_):
        area1 = area_[idx_] * T.prod(resolution_[idx_])
        area2 = area_[idx_ + 1] * T.prod(resolution_[idx_ + 1])
        h = location_[idx_ + 1] - location_[idx_]
        volume = (area1 + area2 )* np.prod(fixed_size).astype('float32') * h / 2.0 / 1000
        return prior_result_ + volume

    predict_V_list, _ = theano.scan(fn=_step,
                              outputs_info = np.array([0.]).astype('float32'),
                              sequences = T.arange(1000),
                              non_sequences = [area, location, resolution],
                              n_steps = location.shape[0] - 1)
    predict_V = predict_V_list[-1]
    return predict_V[0]

def stage3_load_single_record(file_path, fixed_size):
    data = np.load(file_path).item()
    patch_list = data['patchStack']
    location_list = np.array(data['SliceLocation'])
    resolution = np.array(data['PixelSpacing'])
    resized_resolution_list = []
    resized_patch_list = []
    for patch in patch_list:
        resized_resolution_list.append(
            (resolution[0] / fixed_size[0] * patch.shape[0], resolution[1] / fixed_size[1] * patch.shape[1]))
        resized_patch_list.append(cv2.resize(patch, fixed_size))
    
    resized_patch_list = np.array(resized_patch_list, dtype='float32')[:, None, :, :]
    location_list = np.array(location_list, dtype='float32')
    resized_resolution_list = np.array(resized_resolution_list, dtype='float32')
    return resized_patch_list, location_list, resized_resolution_list

# Configuration

In [3]:
fixed_size = (48, 48)

# Load data

In [4]:
# read train and val volume data
volume_csv_path = '../../clean/stage3/train.csv'
volume_csv = pd.read_csv(volume_csv_path)
volume_data = np.array(volume_csv.iloc[:, 1:3])
rows = volume_data.shape[0]
volume_data = np.repeat(volume_data.flatten(), 4).astype('float32')

# read train and val patch, location and resolution
root_dir = '../../clean/stage3'
min_root_dir = os.path.join(root_dir, 'min')
max_root_dir = os.path.join(root_dir, 'max')
x_data = []
location_data = []
resolution_data = []
for i in range(1, rows + 1):
    min_full_path = os.path.join(min_root_dir, str(i) + '.npy')
    max_full_path = os.path.join(max_root_dir, str(i) + '.npy')
    paths = [min_full_path, max_full_path]
    for path in paths:
        x_data_single, location_data_single, resolution_data_single = stage3_load_single_record(path, fixed_size)
        x_data.append(x_data_single)
        location_data.append(location_data_single)
        resolution_data.append(resolution_data_single)

# Build model

In [5]:
location = T.vector('location')
resolution = T.matrix('resolution')
target_volume = T.fscalar('volume')

# all adapters
adapters = []
adapters.append(adapter1((48, 48), '96.npz'))

# input tensor
pred = T.tensor4('pred')
area = T.mean(pred, axis=[1, 2, 3])
pred_volume = build_volume(area, location, resolution)
loss = T.abs_(pred_volume - target_volume).mean() / 600

val_fn = theano.function(
    [pred, location, resolution, target_volume],
    loss
)

ValueError: snapshot 96.npz not found

# Training and Validating

In [ ]:
print("-----------------predicting---------------------")
test_losses = []
for j in range(len(x_data)):
    x_e = x_data[j].astype('float32')
    preds = []
    for adapter in adapters:
        preds.append(adapter.convert(x_e))
    pred_e = np.concatenate(preds, axis=1)
    location_e = location_data[j].astype('float32')
    resolution_e = resolution_data[j].astype('float32')
    volume_e = volume_data[j].astype('float32')
    loss = val_fn(pred_e, location_e, resolution_e, volume_e)
    test_losses.append(loss)
print("total #samples: {}, loss: {}".format(len(x_data), np.mean(test_losses)))